In [1]:
using FunctionalCollections
using Iterators
using Pipe
using Compat

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")

 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in require at ./loading.jl:243
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:177
 in eventloop at /home/ubuntu/.julia/v0.5/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:443
while loading /home/ubuntu/.julia/v0.5/FunctionalCollections/src/PersistentArrayMap.jl, in expression starting on line 58
 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in require at ./loading.jl:243
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:177
 in eventloop at /home/ubuntu/.julia/v0.5/IJulia

4-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              
 "../util/"                                                       

 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in require at ./loading.jl:243
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:177
 in eventloop at /home/ubuntu/.julia/v0.5/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:443
while loading /home/ubuntu/.julia/v0.5/FunctionalCollections/src/PersistentSet.jl, in expression starting on line 47
 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in require at ./loading.jl:243
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:177
 in eventloop at /home/ubuntu/.julia/v0.5/IJulia/src/

In [2]:
const START_MARKER1 = "**START1**"
const START_MARKER2 = "**START2**"
const END_MARKER1 = "**END1**"
const END_MARKER2 = "**END2**"

atis_data = open("atis_data2.jsz","r") do fh
    deserialize(fh)    
end


Dict{ASCIIString,Any} with 5 entries:
  "length_prob"   => [0.00274146,0.0142225,0.0332953,0.0543865,0.0725589,0.0850…
  "LL"            => 300x1120 Array{Float64,2}:…
  "indexed_words" => AbstractString["in","for","that","is","on","with","said","…
  "trigrams"      => Any[("**START1**","**START2**","what"),("**START2**","what…
  "word_indexes"  => Dict{AbstractString,Int64}("carried"=>502,"diabetic"=>927,…

In [3]:
using PyCall
#http://www.nltk.org/howto/probability.html
@pyimport nltk
@pyimport nltk.probability as nltk_prob


kn_prob_dist = nltk_prob.KneserNeyProbDist(pycall(nltk_prob.FreqDist, PyObject, atis_data["trigrams"]))
function trigram_model(given1::AbstractString, given2::AbstractString, event::AbstractString)
    kn_prob_dist[:prob]((given1, given2, event))
end

/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/draw/__init__.py:15: UserWarning: nltk.draw package not loaded (please install Tkinter library).
  warnings.warn("nltk.draw package not loaded "


trigram_model (generic function with 1 method)

In [9]:
example1 = ["show", "cheapest", "flight"]
example2 = ["francisco","washington","flight","from","cheapest","san","show"]
example3 = [example2..., "to"]

8-element Array{ASCIIString,1}:
 "francisco" 
 "washington"
 "flight"    
 "from"      
 "cheapest"  
 "san"       
 "show"      
 "to"        

In [5]:
typealias S ASCIIString
typealias State{T} Tuple{T,T}

function transition_prob(cur_state::State{S}, next_word::S)
    trigram_model(cur_state..., next_word)
end

function get_options(cur_state::State{S})
    tp = transition_prob(cur_state, END_MARKER1)
    # Given P(END_MARKER2 | curstatep[2]==END_MARKER1) = 1.0
    # Do not need to consider P(END_MARKER2 | curstatep[2]==END_MARKER1, curstatep[1])
    [
        (EmptyList{S}(), tp)
    ]
end

function _get_options(cur_state::State{S}, remaining_words::S...)
    collect(@task begin
        for ii in 1:length(remaining_words)
            word = remaining_words[ii]
            tp = transition_prob(cur_state, word) 
            next_state = (cur_state[2], word)

            new_remaining_words = remaining_words[[1:ii-1; ii+1:end]]

            for (tail, next_tp) in get_options(next_state, new_remaining_words...)
                produce(cons(word, tail),tp*next_tp)
            end
        end
    end)
end


_get_options_cache = Dict{Tuple{State{S}, Any}, Vector{Tuple{plist{S}, Float64}}}()
function get_options(state::State{S}, remaining_words::S...)
    if haskey(_get_options_cache, (state, remaining_words))
        _get_options_cache[ (state, remaining_words)]
    else 
        _get_options_cache[(state, remaining_words)] = _get_options(state, remaining_words...)
    end
end

function get_options(remaining_words::S...)
    initial_state = (START_MARKER1, START_MARKER2)
    get_options(initial_state, remaining_words...)
end

get_options (generic function with 3 methods)

In [6]:
function order(unordered_words)
    orders_and_probs = get_options(unordered_words...)
    best_index = findmax(map(op->op[2], orders_and_probs))[2]
    ordered_words = orders_and_probs[best_index][1]
    
end

order (generic function with 1 method)

In [13]:
order(example1)

ASCIIString(show, cheapest, flight)

In [ ]:
NTuple{ASCIIString,ASCIIString,ASCIIString}

In [ ]:
super(Tuple{Int})Tul